In [1]:
# Instalar bibliotecas
# Instalar Librosa (essencial para áudio)

!pip install librosa
!pip install xxd
!pip install numpy
!pip install sklearn
!pip install zipfile
!pip install os
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install keras
!pip install pydub

ERROR: Could not find a version that satisfies the requirement xxd (from versions: none)
ERROR: No matching distribution found for xxd
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [1]:
# Instalando TensorFlow

!pip install tensorflow==2.16.1

In [2]:
# Upgrade ml_dtypes to a compatible version
!pip install --upgrade ml_dtypes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.3 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.16.1 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incom

In [3]:
# CÉLULA 2 - Imports e Carregamento de Dados
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import librosa
from sklearn.model_selection import train_test_split

print("--- Descompactando o arquivo data.zip ---")
# Certifique-se de que seu data.zip foi carregado no Colab
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')
print("--- Arquivos descompactados. ---")

DATASET_PATH = 'dataset/data'
KEYWORDS = ['ligar', 'desligar', 'noise'] # Verifique se corresponde às suas pastas

--- Descompactando o arquivo data.zip ---
--- Arquivos descompactados. ---


In [4]:
# CÉLULA 3 - Pré-processamento + Data Augmentation Simples
import os
import zipfile
import numpy as np
import tensorflow as tf
import librosa
from sklearn.model_selection import train_test_split # Mover import se não estiver em Célula 2

print("\n--- Processando áudios para espectrogramas lineares COM DATA AUGMENTATION ---")

# --- DEFINIÇÕES ---
N_SAMPLES = 16000
SPECTROGRAM_ROWS = 32
SPECTROGRAM_COLS = 32
FFT_SAMPLES = 256
HOP_LENGTH = N_SAMPLES // SPECTROGRAM_COLS
N_BINS_TO_KEEP = FFT_SAMPLES // 2 + 1

DATASET_PATH = 'dataset/data'
KEYWORDS = ['ligar', 'desligar', 'noise']

# --- FUNÇÕES DE DATA AUGMENTATION ---
def add_noise(y, noise_level=0.005):
    """Adiciona ruído gaussiano aleatório ao sinal de áudio."""
    noise_amp = noise_level * np.random.uniform() * np.amax(y)
    # Garante que o ruído tenha o mesmo tamanho que o áudio
    noise = noise_amp * np.random.normal(size=y.shape[0])
    y_noisy = y + noise
    return y_noisy

def time_shift(y, shift_max=0.1): # Desloca até 10% do áudio
    """Desloca o áudio no tempo por uma quantidade aleatória."""
    shift_amount = int(sr * shift_max * np.random.uniform(low=-1.0, high=1.0))
    y_shifted = np.roll(y, shift_amount)
    # Zera o início/fim se o deslocamento foi grande
    if shift_amount > 0:
        y_shifted[:shift_amount] = 0
    else:
        y_shifted[shift_amount:] = 0
    return y_shifted

# --- FUNÇÃO DE PRÉ-PROCESSAMENTO (AGORA APLICA AUGMENTATION) ---
def get_augmented_linear_spectrogram(wav_file, keyword):
    """Carrega áudio, aplica aumentação aleatória e gera espectrograma linear."""
    global sr # Necessário para time_shift
    try:
        y, sr = librosa.load(wav_file, sr=16000)
        if len(y) < N_SAMPLES: y = np.pad(y, (0, N_SAMPLES - len(y)))
        else: y = y[:N_SAMPLES]

        # --- APLICA AUGMENTATION ALEATÓRIA (Exceto para 'noise') ---
        if keyword != 'noise':
            if np.random.rand() < 0.6: # 60% de chance de adicionar ruído
                y = add_noise(y)
            if np.random.rand() < 0.4: # 40% de chance de deslocar no tempo
                y = time_shift(y)
        # -----------------------------------------------------------

        stft_result = librosa.stft(y, n_fft=FFT_SAMPLES, hop_length=HOP_LENGTH)
        magnitude = np.abs(stft_result)
        log_magnitude = librosa.amplitude_to_db(magnitude, ref=np.max)
        spec_data = log_magnitude

        if spec_data.shape[1] > SPECTROGRAM_COLS:
            spec = spec_data[:N_BINS_TO_KEEP, :SPECTROGRAM_COLS]
        else:
            spec = np.pad(spec_data[:N_BINS_TO_KEEP, :],
                          ((0,0), (0, SPECTROGRAM_COLS - spec_data.shape[1])))

        resized_spec = tf.image.resize(spec[..., np.newaxis],
                                       [SPECTROGRAM_ROWS, SPECTROGRAM_COLS],
                                       method=tf.image.ResizeMethod.BILINEAR).numpy()

        return resized_spec.squeeze()

    except Exception as e:
        print(f"  Erro processando {os.path.basename(wav_file)}: {e}")
        return None

# --- LOOP PRINCIPAL (Chama a função com aumentação) ---
spectrograms = []
labels = []
print("Iniciando processamento dos arquivos WAV com Data Augmentation...")
# Vamos gerar MAIS amostras usando aumentação
TARGET_SAMPLES_PER_CLASS = 150 # Aumentamos o alvo para ter mais dados aumentados

for i, keyword in enumerate(KEYWORDS):
    path = os.path.join(DATASET_PATH, keyword)
    print(f"  Processando pasta: {path}")
    if not os.path.exists(path):
        print(f"  AVISO: Pasta não encontrada.")
        continue

    original_files = [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith('.wav')]
    if not original_files:
        print("    Nenhum arquivo .wav encontrado.")
        continue

    file_count = 0
    # Processa cada arquivo original E cria versões aumentadas até atingir o alvo
    while file_count < TARGET_SAMPLES_PER_CLASS:
        # Pega um arquivo original aleatoriamente
        filepath = np.random.choice(original_files)
        # Gera o espectrograma (que aplica aumentação interna aleatória)
        spec = get_augmented_linear_spectrogram(filepath, keyword)
        if spec is not None:
            spectrograms.append(spec)
            labels.append(i)
            file_count += 1

    print(f"    {file_count} espectrogramas (originais + aumentados) gerados.")

# --- Verificação Final ---
if not spectrograms:
     print("\nERRO: Nenhum espectrograma foi gerado...") # Mensagem de erro
else:
    X = np.array(spectrograms)
    y = np.array(labels)
    X = X[..., np.newaxis]
    print(f"\n--- Processamento concluído. Formato final dos dados: {X.shape} ---")
    # Agora X contém ~450 amostras (150 por classe)
    # Prossiga para a Célula 4 (Treinamento)


--- Processando áudios para espectrogramas lineares COM DATA AUGMENTATION ---
Iniciando processamento dos arquivos WAV com Data Augmentation...
  Processando pasta: dataset/data/ligar
    150 espectrogramas (originais + aumentados) gerados.
  Processando pasta: dataset/data/desligar
    150 espectrogramas (originais + aumentados) gerados.
  Processando pasta: dataset/data/noise
    150 espectrogramas (originais + aumentados) gerados.

--- Processamento concluído. Formato final dos dados: (450, 32, 32, 1) ---


In [5]:
# CÉLULA 4 - Divisão de Dados e Treinamento do Modelo (Corrigida)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

input_shape = X_train.shape[1:]
model = models.Sequential([
    layers.Input(shape=input_shape),
    #layers.AveragePooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'),
    layers.Conv2D(8, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)), # Primeira camada de Pooling - OK
    layers.Conv2D(16, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)), # CORREÇÃO: Removemos a segunda camada de Pooling
    layers.Flatten(),
    layers.Dense(len(KEYWORDS), activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Imprime um resumo da arquitetura para vermos as dimensões
model.summary()

print("\n--- Iniciando o treinamento do modelo ---")
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
print("--- Treinamento concluído! ---")

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nAcurácia final no teste: {accuracy * 100:.2f}%")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2704)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │         8,115 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,363 (36.57 KB)

 Trainable params: 9,363 (36.57 KB)

 Non-trainable params: 0 (0.00 B)


--- Iniciando o treinamento do modelo ---
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 0.3544 - loss: 14.6362 - val_accuracy: 0.3667 - val_loss: 7.2230
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.3351 - loss: 6.0835 - val_accuracy: 0.4000 - val_loss: 1.6782
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4819 - loss: 1.6086 - val_accuracy: 0.5556 - val_loss: 1.0230
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5993 - loss: 0.9958 - val_accuracy: 0.6778 - val_loss: 0.9253
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.6427 - loss: 0.8233 - val_accuracy: 0.6333 - val_loss: 0.7019
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.7400 - loss: 0.7214 - val_accuracy: 0.7667 - val_loss: 0.6561
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.7836 - loss: 0.5702 - val_accuracy: 0.8111 - val_loss: 0.5337
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8

In [6]:
# Conversão para TFLite (Float32 - SEM Quantização)
print("\n--- Convertendo o modelo para o formato TFLite (Float32) ---")

# Salva o modelo primeiro
model_save_path = 'keyword_model_saved'
model.export(model_save_path)

# Conversão simples, sem otimizações de quantização
converter = tf.lite.TFLiteConverter.from_saved_model(model_save_path)
tflite_model = converter.convert()

with open('keyword_model_float.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Modelo Float32 salvo como keyword_model_float.tflite")


--- Convertendo o modelo para o formato TFLite (Float32) ---
Saved artifact at 'keyword_model_saved'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  135389879138064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135390008186192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135389848948752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135389863577744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135389848960272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135389848948560: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Modelo Float32 salvo como keyword_model_float.tflite


In [7]:
# - Converter para um Array em C (Versão Simplificada)
import subprocess
import os

print("\n--- Convertendo para um arquivo C header (.h) ---")

# Nome do arquivo TFLite gerado na célula anterior
tflite_filename = 'keyword_model_float.tflite'
# Nome do arquivo C header de saída
output_h_filename = 'model.h'
# Nome da variável C que conterá o array (DEVE ser g_model para o código C++)
c_array_name = 'g_model'

# Comando xxd para gerar diretamente o array C
command = f"xxd -i {tflite_filename} | sed 's/unsigned char.*/const unsigned char {c_array_name}[] = {{/' > {output_h_filename}"

# Executa o comando
try:
    print(f"Executando comando: {command}")
    subprocess.run(command, shell=True, check=True, capture_output=True, text=True)

    # Adiciona a linha do tamanho ao final do arquivo
    with open(output_h_filename, "a") as f:
        f.write(f"\nconst unsigned int {c_array_name}_len = sizeof({c_array_name});\n")

    print(f"\n✅ Arquivo {output_h_filename} (Float32) criado com sucesso! Faça o download.")

except subprocess.CalledProcessError as e:
    print(f"Erro ao executar xxd/sed: {e}")
    print(e.stderr)
except Exception as e:
     print(f"Um erro inesperado ocorreu: {e}")


--- Convertendo para um arquivo C header (.h) ---
Executando comando: xxd -i keyword_model_float.tflite | sed 's/unsigned char.*/const unsigned char g_model[] = {/' > model.h

✅ Arquivo model.h (Float32) criado com sucesso! Faça o download.
